In [ ]:
import zipfile
with zipfile.ZipFile("./YouTube-music-video-5M.zip", 'r') as zip_ref:
    zip_ref.extractall(".")

In [ ]:
import os
dir_to_files = "./YouTube-music-video-5M-master/youtube_ids"
dataset_dir = "./dataset"

if not os.path.exists(dataset_dir):
    os.mkdir(dataset_dir)
# exteract_feature_queue = []
# ids = []

In [ ]:
import os
files = [dir_to_files+'/'+path for path in os.listdir(dir_to_files)]
files

['./YouTube-music-video-5M-master/youtube_ids/youtube_video_ids_15_250890.txt',
 './YouTube-music-video-5M-master/youtube_ids/youtube_video_ids_04_249301.txt',
 './YouTube-music-video-5M-master/youtube_ids/youtube_video_ids_09_251585.txt',
 './YouTube-music-video-5M-master/youtube_ids/youtube_video_ids_12_248831.txt',
 './YouTube-music-video-5M-master/youtube_ids/youtube_video_ids_03_244387.txt',
 './YouTube-music-video-5M-master/youtube_ids/youtube_video_ids_07_255766.txt',
 './YouTube-music-video-5M-master/youtube_ids/youtube_video_ids_01_235614.txt',
 './YouTube-music-video-5M-master/youtube_ids/youtube_video_ids_06_247838.txt',
 './YouTube-music-video-5M-master/youtube_ids/youtube_video_ids_08_252801.txt',
 './YouTube-music-video-5M-master/youtube_ids/youtube_video_ids_20_195192.txt',
 './YouTube-music-video-5M-master/youtube_ids/youtube_video_ids_17_247362.txt',
 './YouTube-music-video-5M-master/youtube_ids/youtube_video_ids_10_252204.txt',
 './YouTube-music-video-5M-master/youtub

In [ ]:
# import shutil

# shutil.rmtree('./audio')

In [ ]:
import re

def get_music_ids(file_path):
  """
  get all music ids of a file based on below pattern
  # new artist: Drake 3TVXtAsR1Inumwj472S9r4

   3t195yz9xCc
   VkXjvHfP3MM
   7LnBvuzjpr4
   1Ldzm7KGECI
   HL1UzIK-flA
   3XR5mhXtpXw
   WsPfSXJaelk
  """
  regex = r"\n(?![#])(.+)"

  f = open(file_path, 'r')
  return re.findall(regex, f.read())

In [ ]:
len(get_music_ids(files[1]))

249301

In [ ]:
!pip install youtube_dl

In [ ]:
from __future__ import unicode_literals
import youtube_dl


def download_music(music_urls: list):
  """
  download a list of youtube urls
  """

  ydl_opts = {
      # 'proxy': 'socks5://5.252.161.48:1080',
      'force-ipv4': True,
      "external-downloader": "aria2c",
      "external-downloader-args": "-x 16 -s 16 -k 1M",
      'ignoreerrors': True,
      'format': 'bestaudio/best',
      'extractaudio' : True,      # only keep the audio
      'audioformat' : "wav",      # convert to mp3 
      'outtmpl': './audio/%(id)s.wav',        # name the file the ID of the video
      'noplaylist' : True,
      'max-filesize': '10m',
      'quiet': True,
      'no-warnings': True
  }
  with youtube_dl.YoutubeDL(ydl_opts) as ydl:
      ydl.download(music_urls)

In [ ]:
def get_youtube_urls(music_ids: list) -> list:
  """
  get id and return youtube url
  example:
  7ZUMRECYLOQ => https://www.youtube.com/watch?v=7ZUMRECYLOQ
  """
  base_url = "https://www.youtube.com/watch?v={}"
  return [base_url.format(music_id) for music_id in music_ids]

In [ ]:
sample_rate = 16000
hop_length = int(sample_rate * 0.1)
n_fft = int(sample_rate * 0.2)

In [ ]:
import librosa
import numpy as np

def get_audio_features(file_path: str) -> np.ndarray:
  """
  extract features from audio file
  """
  try:
    x, sr = librosa.load(file_path, sample_rate, res_type="kaiser_fast")
    chromagram = librosa.feature.chroma_stft(x, sr=sr, n_fft=n_fft, 
                                            hop_length=hop_length)
  except Exception as e:
      print(e)
      return None
  os.remove(file_path)
  return chromagram.transpose()

In [ ]:
import time

def check_limits_for_download(music_urls: list) -> list:
  """
  if the size or duration is ok it is allowed to download
  return allowed list
  """
  MAX_DURATION = 600
  MAX_SIZE = 10485760 # 10M

  allowed = list()
  for url in music_urls:
    audio_downloder = youtube_dl.YoutubeDL({'format':'bestaudio', 
                                            "external-downloader": "aria2c",
                                            "external-downloader-args": "-x 16 -s 16 -k 1M",
                                            'ignoreerrors': True,
                                            'quiet': True, 
                                            'no-warnings': True})
    info = audio_downloder.extract_info(url, download=False) 
    if info and info.get('duration', float('inf')) <= MAX_DURATION and info.get('filesize', float('inf')) <= MAX_SIZE:
      allowed.append(url)
    if not info:
      time.sleep(0.2)
  return allowed

  

In [ ]:
def block_management(music_ids: list) -> list:
  """
  get url of ids + download them as audio + extract features
  """
  allowed = check_limits_for_download(get_youtube_urls(music_ids))
  download_music(allowed)
  features = list()
  for each in music_ids:
    result = get_audio_features(f"./audio/{each}.wav")
    if result is not None:
      features.append(result)
  print(f'+++++++++++++++++++++++++++++++++++over=>{len(features)}element, allowed: {len(allowed)}+++++++++++++++++++++++++++++')
  return features

In [ ]:
music_ids_list = [each for path in files for each in get_music_ids(path)]

len(music_ids_list)

5081000

In [ ]:
MAX_NUM_PROCESS = 10
MAX_NUM_ELEMENT_BLOCK = 20
MAX_NUM_IDS = 200

In [ ]:
def save_array(file_name: str, array: np.array, path=dataset_dir):
  with open(f'{path}/{file_name}.npy', 'wb') as f:
    np.save(f, array)

In [ ]:
from multiprocessing import Pool
pool = Pool(MAX_NUM_PROCESS)

for file_id in range(1550, 1560):
  each = music_ids_list[file_id * MAX_NUM_IDS: (file_id + 1) * MAX_NUM_IDS]
  args = [each[(i * MAX_NUM_ELEMENT_BLOCK): ((i + 1) * MAX_NUM_ELEMENT_BLOCK)] 
          for i in range(int(len(each)/MAX_NUM_ELEMENT_BLOCK)+1)]
  results = pool.map(block_management, args)
  save_array(str(file_id), [each for result in results for each in result])

ERROR: Video unavailable
This video contains content from SBS(Seoul Broadcasting System), who has blocked it on copyright grounds.
ERROR: No video formats found; please report this issue on https://yt-dl.org/bug . Make sure you are using the latest version; see  https://yt-dl.org/update  on how to update. Be sure to call youtube-dl with the --verbose flag and include its complete output.
ERROR: No video formats found; please report this issue on https://yt-dl.org/bug . Make sure you are using the latest version; see  https://yt-dl.org/update  on how to update. Be sure to call youtube-dl with the --verbose flag and include its complete output.
ERROR: No video formats found; please report this issue on https://yt-dl.org/bug . Make sure you are using the latest version; see  https://yt-dl.org/update  on how to update. Be sure to call youtube-dl with the --verbose flag and include its complete output.
ERROR: No video formats found; please report this issue on https://yt-dl.org/bug . Make s

[Errno 2] No such file or directory: './audio/ZxsaYVNkosQ.wav'


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


[Errno 2] No such file or directory: './audio/et-4L_JOyGk.wav'


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


...

[Errno 2] No such file or directory: './audio/d5WqSrcS8qY.wav'


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


[Errno 2] No such file or directory: './audio/N0SxT3sRLY4.wav'


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


[Errno 2] No such file or directory: './audio/ROSo6UZQJMo.wav'
+++++++++++++++++++++++++++++++++++over=>7element, allowed: 7+++++++++++++++++++++++++++++
